# Mapping accuracy and runtime using strobeAlign

## StrobeAlign is a new short read aligner designed to be optimum for sequence reads of length 100, 200, 300, 400, 500 bps.
## This is highlighted as being the optimal range of base pair alignment for this aligner.
The first step of calculating these accuracies for the full genome will be as follow:
We have a complete human genome (Genome assembly T2T-CHM13v2.0) of size 3.0 Gb, to cover the full genome with 100 bp reads we will need about 30 million reads (since 3.0 Gb / 100 bp = 30 million reads).
We can optain around the same number of reads using -fcov 1 and -cov 1,250,000 using the read simulator ART (Let's use -cov).

#### The direction below can be used to test the runtime and accuracy of single-end and paired-end reads using StrobeAlign for the whole reference genome. 

#### Further down, there will be a Trial section where I tested smaller reads (24,000 reads) since 30 million reads was crashing my machines.

#### Parameter definition 

-ef  --errfree  indicate to generate the zero sequencing errors SAM file as well the regular one NOTE: the reads in the zero-error SAM file have the same alignment positions as those in the regular SAM file, but have no sequencing errors

StrobeAlign can be run either by generating the index on the fly or before the alignment is ran.

coverage = number of reads/read pairs to be generated per sequence/amplicon (not be used together with -f/--fcov)
The coverage number i.e 1,250,000 for 100 length reads calculated by dividing 30 million reads / 24 sequences in the reference genome.

Indels (-maxIndel) =  The parameter used to set the maximum length for indel errors. Indels, insertions and deletions, are a type of sequencing error where bases are erroneously inserted into or deleted from the sequence during the sequencing process. 

In [1]:
try:
    import os
    import time
    from Bio import SeqIO
    from Bio.Seq import Seq
    import subprocess
    import tempfile
    import pysam
    import matplotlib.pyplot as plt
    import itertools
except:
    print("One or more required packages is not installed.")
    print("Please verify dependencies and try again.")
    exit()

In [2]:
os.getcwd()

'/home/lajoyce/Desktop'

## Reads from the Whole Genome

### Reads can be simulated using the ART software. My Trial section has example of command lines I used for these tests. However, below are the numbers that should be used for the overall tests for the whole genome.

##### Single-end reads generation command example.

art_illumina -ss HS25 -sam -i /home/lajoyce/Documents/sequence_alignment/complete_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna -l 100 -c 10000 -ef -rs 10 -o /home/lajoyce/Documents/sequence_alignment/complete_genome/trial/100_long_single_reads_cov

##### The -c and -l parameter should be replaced with proper values that represent the experimental environment desired such that:

##### -l 100 = -c 1,250,000 (generates 30 million single reads)
##### -l 200 = -c 625,000 (generates 15 million single reads)
##### -l 300 = -c 416,666.667 (generates 10 million single reads)
.
.
.

##### Paired-end reads generation command example.

art_illumina -ss HS25 -i GCF_009914755.1_T2T-CHM13v2.0_genomic.fna -p -l 100 -c 1250000 -m 300 -s 20 -ef -o 100_paired_end_reads

If -m 300 -s 20 it means the simulated paired-end reads will come from fragments with an average size of 300 pairs, and most of the fragments will fall within 20 base pairs of this average (that is, between 280 and 320 base pairs). This is because in a normal distribution, most values (68%) fall within one standard deviation from the mean.


##### The download and instructions on how to use StrobeAlign can be found on the link below.

https://github.com/ksahlin/strobealign

In short, the index matrix used by Strobe can be generated on the flight or by creating the index matrix separately (then add --use-index parameter.)

### For sanity check we can check that the reads generated in the fastq file is the same as the reads in the error free sam files.

In [ ]:
# Read the FASTQ file into a dictionary
fastq_dict = SeqIO.to_dict(SeqIO.parse("/home/lajoyce/Documents/sequence_alignment/complete_genome/single_end_reads/100_long_single_reads_cov.fq", "fastq"))

# Open the SAM file
samfile = pysam.AlignmentFile("/home/lajoyce/Documents/sequence_alignment/complete_genome/single_end_reads/100_long_single_reads_cov_errFree.sam", "r")

# Iterate over the reads in the SAM file
for read in samfile:
    #Get the read indentifier
    read_id = read.query_name

    #Check if the read identifier exists in the FASTQ dictionary
    if read_id in fastq_dict:
        # Compare the sequences

        if read.query_sequence == str(fastq_dict[read_id].seq):
            print(f"Read {read_id} matches.")
        else:
            print(f"Read {read_id} does not match.")
    else:
        print(f"Read {read_id} not found in FASTQ file.")

# Close the SAM file
samfile.close()        

### You can prepare the other reads from the terminal or other methods by changing the length of the reads.

##### Running the cell below after generating all the reads with different length should print out the runtimes for all the different reads. The same idea can be followed to get the accuracy for the single and paired end reads too.

In [ ]:

reads_names = ["100_long_single_reads_cov.fq", "200_long_single_reads_cov.fq", "300_long_single_reads_cov.fq", "400_long_single_reads_cov.fq", "500_long_single_reads_cov.fq"]


counter = 0 

runtimes = []

for reads in reads_names:

    reads_file = f"/home/lajoyce/Documents/sequence_alignment/complete_genome/single_end_reads/{reads}" 

    reference_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna"

#output_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/sam_files/100_single_ends_strobe.sam"



# Define the StrobeAlign command 
# If you want to run strobealign without the predefined index, don't add the "--use-index" parameter
    strobealign_command = ["/home/lajoyce/Documents/sequence_alignment/strobealign/build/strobealign", "--use-index", reference_file, reads_file]

# Start the timer

    start_time = time.time()

    #Run StrobeAlign and discard the output since we may only care about about the runtime

    subprocess.run(strobealign_command, stdout=subprocess.DEVNULL, check = True)

    # Stop the timer
    end_time = time.time()

    # Delete the temporary FASTQ file
    #os.remove(temp_file_name)

    # Calculate and return the runtime 
    runtime = end_time - start_time

    runtimes.append(runtime)

    print(f"Runtime: {runtime} seconds")

    counter += 1

    print(counter)

print(runtimes)

# Trial (Overall BWA-MEM is more accurate than StrobeAlign)

# I had to use smaller reads instead of 30 million to avoid my systems crashing. In general, StrobeAlign uses a lot of RAM to generate the index and obviously running 30 million reads for processing is also memory intensive

Adding the errorfree (-ef) parameter generates an additional errorfree sam file that we will use as the ground truth to compare the alignments, especially for accuracy. 

### Art_illumina command

art_illumina -ss HS25 -sam -i /home/lajoyce/Documents/sequence_alignment/complete_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna -l 100 -c 10000 -ef -rs 10 -o /home/lajoyce/Documents/sequence_alignment/complete_genome/trial/100_long_single_reads_cov

### StrobeAlign command
./build/strobealign --use-index /home/lajoyce/Documents/sequence_alignment/complete_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna /home/lajoyce/Documents/sequence_alignment/complete_genome/trial/100_long_single_reads_cov.fq > /home/lajoyce/Documents/sequence_alignment/complete_genome/trial/100_single_ends_strobe.sam

In [ ]:
try:
    import os
    import time
    from Bio import SeqIO
    import subprocess
    import tempfile
    import pysam
except:
    print("One or more required packages is not installed.")
    print("Please verify dependencies and try again.")
    exit()

os.getcwd()

### For sanity check we can check that the reads generated in the fastq file is the same as the reads in the error free sam files and that the standard sam file is different from the errorfree sam files. 

##### Conclusion: As expected, the reads in each sam files are not the same as the standard sam file includes errors from the Illumina machine that it is simulating while the errorfree sam file does not. 

##### The reads from the fastq file matches the reads from the standard sam file (these reads are reverse complemented.)

##### The code below checks if the reads from the fastq files are the same as the reads from the standard/error free sam files

In [ ]:
# Load the FASTQ reads into a set 

fastq_reads = set()
for record in SeqIO.parse("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov.fq", "fastq"):
    fastq_reads.add(str(record.seq))


# Open the SAM file and compare each read
samfile = pysam.AlignmentFile("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov.sam", "r")

# Open the errorfree SAM file and compare each read to the fastq file
#samfile = pysam.AlignmentFile("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov_errFree.sam", "r")

count = 0
total_reads = 0
for read in samfile:
    seq = read.query_sequence
    # If the read is reverse complemented (flag 16), reversese complement the sequence before comparison
    if read.is_reverse:
        seq = str(Seq(seq).reverse_complement())
    if seq in fastq_reads:
        count += 1
    total_reads += 1

print(f"Number of matching reads: {count}")

if count == total_reads:
    print("All reads matched.")
else:
    print("Not all reads matched.")


samfile.close()

##### The code below checks if the standard sam file has the same reads as the errorfree sam file.

In [ ]:
# CIGARString comparison

def compare_sam_files(file1, file2):
    samfile1 = pysam.AlignmentFile(file1, "r")
    samfile2 = pysam.AlignmentFile(file2, "r")

    for read1, read2 in zip(samfile1, samfile2):
        # Compare essential fields
        if (read1.query_name != read2.query_name or read1.reference_start != read2.reference_start or read1.reference_end != read2.reference_end or read1.cigarstring != read2.cigarstring):
            return False
    return True

same = compare_sam_files("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov.sam", "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov_errFree.sam")

print("SAM files are the same: ", same)

In [ ]:
# Check sequences between both sam files

def compare_sam_files_sequences(file1, file2):
    samfile1 = pysam.AlignmentFile(file1, "r")
    samfile2 = pysam.AlignmentFile(file2, "r")

    for read1, read2 in zip(samfile1, samfile2):
        # Reverse complement sequence if needed
        seq1 = read1.query_sequence
        seq2 = read2.query_sequence
        if read1.is_reverse:
            seq1 = str(Seq(seq1).reverse_complement())
        if read2.is_reverse:
            seq2 = str(Seq(seq2).reverse_complement())
        
        # Compare sequences
        if seq1 != seq2:
            return False
    return True

same_sequences = compare_sam_files_sequences("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov.sam", "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov_errFree.sam")

print("SAM files have the same sequences: ", same_sequences)


##### The code below checks if the starting positions for both sam files are the same

In [ ]:
samfile1 = pysam.AlignmentFile("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov.sam", "r")
samfile2 = pysam.AlignmentFile("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov_errFree.sam", "r")

total_reads = 0
match_count = 0

for read1, read2 in zip(samfile1.fetch(until_eof=True), samfile2.fetch(until_eof=True)):

    total_reads += 1
    # Compare start positions
    if read1.reference_start == read2.reference_start:
        match_count += 1

print(f"Number of matching start positions: {count}")

#Check if all the starting positions matched
if total_reads == match_count:
    print("All starting positions matched.")
else:
    print("Not all starting positions matched.")

samfile1.close()
samfile2.close()

# Runtime Single End Reads

In [ ]:
reads_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/100_single_end/100_long_single_reads_cov.fq" 

reference_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna"

#output_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/sam_files/100_single_ends_strobe.sam"



# Define the StrobeAlign command 
# If you want to run strobealign without the predefined index, don't add the "--use-index" parameter
#strobealign_command = ["/home/lajoyce/Documents/sequence_alignment/strobealign/build/strobealign", "--use-index", reference_file, reads_file]

# Command to use if you want to keep the output of strobealign
#If you want to save the output, add the directory and name of the output file
strobealign_command = ' '.join(['/home/lajoyce/Documents/sequence_alignment/strobealign/build/strobealign', '--use-index', 
                                '/home/lajoyce/Documents/sequence_alignment/complete_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna', 
                                '/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov.fq',
                                  '> /home/lajoyce/Documents/sequence_alignment/complete_genome/trial/strobealign/single_ends_strobe.sam'])

# Start the timer

start_time = time.time()

#try:
#Run StrobeAlign and do not discard the output
subprocess.run(strobealign_command, check=True, shell=True)

#except subprocess.CalledProcessError as e:
#    print("Error:")
 #   print(e.stderr)

#Run StrobeAlign and discard the output since we may only care about about the runtime

#subprocess.run(strobealign_command, stdout=subprocess.DEVNULL, check = True)

# Stop the timer
end_time = time.time()

#Delete the temporary FASTQ file
#os.remove(temp_file_name)

# Calculate and return the runtime 
runtime = end_time - start_time

print(f"Runtime: {runtime} seconds")

# Run time for all the reads (single-end reads)

### The idea is to generate reads with different length and then test it all together and plot the results. This can be done for paired-end reads etc...

In [ ]:

reads_names = ["100_long_single_reads_cov.fq", "200_long_single_reads_cov.fq", "300_long_single_reads_cov.fq", "400_long_single_reads_cov.fq", "500_long_single_reads_cov.fq"]


counter = 0 

runtimes = []

for reads in reads_names:

    reads_file = f"/home/lajoyce/Documents/sequence_alignment/complete_genome/single_end_reads/{reads}" 

    reference_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna"

#output_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/sam_files/100_single_ends_strobe.sam"



# Define the StrobeAlign command 
# If you want to run strobealign without the predefined index, don't add the "--use-index" parameter
    strobealign_command = ["/home/lajoyce/Documents/sequence_alignment/strobealign/build/strobealign", "--use-index", reference_file, reads_file]

# Start the timer

    start_time = time.time()

    #Run StrobeAlign and discard the output since we may only care about about the runtime

    subprocess.run(strobealign_command, stdout=subprocess.DEVNULL, check = True)

    # Stop the timer
    end_time = time.time()

    # Delete the temporary FASTQ file
    #os.remove(temp_file_name)

    # Calculate and return the runtime 
    runtime = end_time - start_time

    runtimes.append(runtime)

    print(f"Runtime: {runtime} seconds")

    counter += 1

    print(counter)

print(runtimes)

## Accuracy & Multi-mapping reading

### Comparing starting positions only

In [ ]:
def calculate_accuracy_and_multimapping(ground_truth_sam, test_sam):
    # Initialize variables
    total_reads = 0
    correct_alignments = 0
    multimapped_reads = 0

    with pysam.AlignmentFile(ground_truth_sam, "r") as gt_sam, pysam.AlignmentFile(test_sam, "r") as t_sam:
        for truth, test in zip(gt_sam, t_sam):
            # Increase total reads
            total_reads += 1
            
            # Check if alignments are the same
            if truth.reference_start == test.reference_start:
                correct_alignments += 1

            # Check if read is multi-mapped
            if test.has_tag("XA"):
                multimapped_reads += 1
                
    # If no reads found, print a message and return
    if total_reads == 0:
        print("No reads found in the SAM files.")
        return 0, 0
    
    # Calculate the accuracy
    accuracy = correct_alignments / total_reads
    multimapping_percentage = multimapped_reads / total_reads
    print(f"The number of correct alignments: {correct_alignments}")

    return accuracy, multimapping_percentage

accuracy, multimapping_percentage = calculate_accuracy_and_multimapping("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov.sam", "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/strobealign/single_ends_strobe.sam")

print(f"Alignment accuracy: {accuracy * 100}%")
print(f"Percentage of multi-mapped reads: {multimapping_percentage * 100}%")


### Check for starting and ending positions

In [ ]:
def calculate_accuracy_and_multimapping(ground_truth_sam, test_sam):
    # Initialize variables
    total_reads = 0
    correct_alignments = 0
    multimapped_reads = 0

    with pysam.AlignmentFile(ground_truth_sam, "r") as gt_sam, pysam.AlignmentFile(test_sam, "r") as t_sam:
        for truth, test in zip(gt_sam, t_sam):
            # Increase total reads
            total_reads += 1

            # Compare start and inferred end positions
            if truth.reference_start == test.reference_start and truth.infer_read_length() == test.infer_read_length():
                correct_alignments += 1

            # Check if read is multi-mapped
            if test.has_tag("XA"):
                multimapped_reads += 1

    # If no reads found, print a message and return
    if total_reads == 0:
        print("No reads found in the SAM files.")
        return 0, 0

    # Calculate the accuracy
    accuracy = correct_alignments / total_reads
    multimapping_percentage = multimapped_reads / total_reads

    return accuracy, multimapping_percentage

# Use the function
accuracy, multimapping_percentage = calculate_accuracy_and_multimapping("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov.sam", "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/strobealign/single_ends_strobe.sam")

print(f"Alignment accuracy: {accuracy * 100}%")
print(f"Percentage of multi-mapped reads: {multimapping_percentage * 100}%")


# Paired-end reads 

### ART can generate paired-end reads.

##### The reads I generated for my trial tests was done by the command below.

In [ ]:
# Parameters
reference_genome = "/home/lajoyce/Documents/sequence_alignment/complete_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna"
output_filename = "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/paired_end_reads/100_paired_end_reads"
read_length = 100  # The desired read length
coverage = 1000  # The desired coverage
insert_size = 300  # The desired mean insert size
insert_std_dev = 20  # The desired standard deviation for the insert size

# Command
command = f"art_illumina -ss HS25 -i {reference_genome} -p -l {read_length} -c {coverage} -m {insert_size} -s {insert_std_dev} -na -ef -o {output_filename}"

# Run the command
process = subprocess.Popen(command, shell=True)

process.wait()


# The separated FASTQ files can be conbined into interleaved fastq files.

In [ ]:
def interleave(iter1, iter2):
    # Interleave the reads from two iterators
    return itertools.chain.from_iterable(itertools.zip_longest(iter1, iter2))

file1 = "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/paired_end_reads/100_paired_end_reads1.fq"
file2 = "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/paired_end_reads/100_paired_end_reads2.fq"
outfile = "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/paired_end_reads/interleaved_reads.fq"

read1 = SeqIO.parse(file1, "fastq")
read2 = SeqIO.parse(file2, "fastq")

interleaved = interleave(read1, read2)

count = SeqIO.write(interleaved, outfile, "fastq")
print("Interleaved %i reads" % count)

#### Paired end reads (interleaved) StrobeAlign

In [ ]:
reads_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/100_single_end/100_long_single_reads_cov.fq" 

reference_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna"

#output_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/sam_files/100_single_ends_strobe.sam"



# Define the StrobeAlign command 
# If you want to run strobealign without the predefined index, don't add the "--use-index" parameter
#strobealign_command = ["/home/lajoyce/Documents/sequence_alignment/strobealign/build/strobealign", "--use-index", reference_file, reads_file]

# Command to use if you want to keep the output of strobealign
#If you want to save the output, add the directory and name of the output file

# To report secondary alignments, set parameter -N [INT] for a maximum of [INT] secondary alignments.
strobealign_command = ' '.join(['/home/lajoyce/Documents/sequence_alignment/strobealign/build/strobealign', '--use-index', 
                                '/home/lajoyce/Documents/sequence_alignment/complete_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna', 
                                '/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/paired_end_reads/interleaved_reads.fq',
                                  '> /home/lajoyce/Documents/sequence_alignment/complete_genome/trial/strobealign/paired_end_reads/100_paired_ends_strobe.sam'])

# Start the timer

start_time = time.time()

#try:
#Run StrobeAlign and do not discard the output
subprocess.run(strobealign_command, check=True, shell=True)

#except subprocess.CalledProcessError as e:
#    print("Error:")
 #   print(e.stderr)

#Run StrobeAlign and discard the output since we may only care about about the runtime

#subprocess.run(strobealign_command, stdout=subprocess.DEVNULL, check = True)

# Stop the timer
end_time = time.time()

#Delete the temporary FASTQ file
#os.remove(temp_file_name)

# Calculate and return the runtime 
runtime = end_time - start_time

print(f"Runtime: {runtime} seconds")

# Accuracy Check Starting Position

In [ ]:
def calculate_accuracy_and_multimapping(ground_truth_sam, test_sam):
    # Initialize variables
    total_reads = 0
    correct_alignments = 0
    multimapped_reads = 0

    with pysam.AlignmentFile(ground_truth_sam, "r") as gt_sam, pysam.AlignmentFile(test_sam, "r") as t_sam:
        for truth, test in zip(gt_sam, t_sam):
            # Increase total reads
            total_reads += 1
            
            # Check if alignments are the same
            if truth.reference_start == test.reference_start:
                correct_alignments += 1

            # Check if read is multi-mapped
            if test.has_tag("XA"):
                multimapped_reads += 1
                
    # If no reads found, print a message and return
    if total_reads == 0:
        print("No reads found in the SAM files.")
        return 0, 0
    
    # Calculate the accuracy
    accuracy = correct_alignments / total_reads
    multimapping_percentage = multimapped_reads / total_reads
    print(f"The number of correct alignments: {correct_alignments}")

    return accuracy, multimapping_percentage

accuracy, multimapping_percentage = calculate_accuracy_and_multimapping("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/paired_end_reads/100_paired_end_reads.sam", "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/strobealign/paired_end_reads/100_paired_ends_strobe.sam")

print(f"Alignment accuracy: {accuracy * 100}%")
print(f"Percentage of multi-mapped reads: {multimapping_percentage * 100}%")


## Accuracy checking for starting and ending positions

In [ ]:
def calculate_accuracy_and_multimapping(ground_truth_sam, test_sam):
    # Initialize variables
    total_reads = 0
    correct_alignments = 0
    multimapped_reads = 0

    with pysam.AlignmentFile(ground_truth_sam, "r") as gt_sam, pysam.AlignmentFile(test_sam, "r") as t_sam:
        for truth, test in zip(gt_sam, t_sam):
            # Increase total reads
            total_reads += 1

            # Compare start and inferred end positions
            if truth.reference_start == test.reference_start and truth.infer_read_length() == test.infer_read_length():
                correct_alignments += 1

            # Check if read is multi-mapped
            if test.has_tag("XA"):
                multimapped_reads += 1

    # If no reads found, print a message and return
    if total_reads == 0:
        print("No reads found in the SAM files.")
        return 0, 0

    # Calculate the accuracy
    accuracy = correct_alignments / total_reads
    multimapping_percentage = multimapped_reads / total_reads

    return accuracy, multimapping_percentage

# Use the function
accuracy, multimapping_percentage = calculate_accuracy_and_multimapping("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/paired_end_reads/100_paired_end_reads.sam", "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/strobealign/paired_end_reads/100_paired_ends_strobe.sam")

print(f"Alignment accuracy: {accuracy * 100}%")
print(f"Percentage of multi-mapped reads: {multimapping_percentage * 100}%")


### Conclusion: Accuracy here should be the same as simply checking the starting positions.

# BWA-MEM Double Check 

### BWA-MEM2 gave me errors when I tried to generate the index for the reference genome with it.

### We are going to use BWA-MEM instead then.

In [ ]:
os.chdir("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/bwa-mem2")

### Test for Runtime

In [ ]:
reads_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/100_single_end/100_long_single_reads_cov.fq" 

reference_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna"

#output_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/sam_files/100_single_ends_strobe.sam"


# Define the StrobeAlign command 
# If you want to run strobealign without the predefined index, don't add the "--use-index" parameter
#strobealign_command = ["/home/lajoyce/Documents/sequence_alignment/strobealign/build/strobealign", "--use-index", reference_file, reads_file]

# Command to use if you want to keep the output of strobealign
#If you want to save the output, add the directory and name of the output file
bwamem2_command = ' '.join(["/home/lajoyce/Documents/sequence_alignment/bwa/bwa",
                            "mem",
                            "GCF_009914755.1_T2T-CHM13v2.0_genomic.fna",
                            "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov.fq",
                             "> /home/lajoyce/Documents/sequence_alignment/complete_genome/trial/bwa-mem2/100_long_single_end_reads.sam"])
                            
# Start the timer

start_time = time.time()

#try:
#Run StrobeAlign and do not discard the output
subprocess.run(bwamem2_command, check=True, shell=True)

#except subprocess.CalledProcessError as e:
#    print("Error:")
 #   print(e.stderr)

#Run StrobeAlign and discard the output since we may only care about about the runtime

#subprocess.run(strobealign_command, stdout=subprocess.DEVNULL, check = True)

# Stop the timer
end_time = time.time()

#Delete the temporary FASTQ file
#os.remove(temp_file_name)

# Calculate and return the runtime 
runtime = end_time - start_time

print(f"Runtime: {runtime} seconds")

### Compare BWA-MEM SAM file to the Ground Truth File for Accuracy

#### Here we compare starting positions only

In [ ]:
def calculate_accuracy_and_multimapping(ground_truth_sam, test_sam):
    # Initialize variables
    total_reads = 0
    correct_alignments = 0
    multimapped_reads = 0

    with pysam.AlignmentFile(ground_truth_sam, "r") as gt_sam, pysam.AlignmentFile(test_sam, "r") as t_sam:
        for truth, test in zip(gt_sam, t_sam):
            # Increase total reads
            total_reads += 1
            
            # Check if alignments are the same
            if truth.reference_start == test.reference_start:
                correct_alignments += 1

            # Check if read is multi-mapped
            if test.has_tag("XA"):
                multimapped_reads += 1
                
    # If no reads found, print a message and return
    if total_reads == 0:
        print("No reads found in the SAM files.")
        return 0, 0
    
    # Calculate the accuracy
    accuracy = correct_alignments / total_reads
    multimapping_percentage = multimapped_reads / total_reads
    print(f"The number of correct alignments: {correct_alignments}")

    return accuracy, multimapping_percentage

accuracy, multimapping_percentage = calculate_accuracy_and_multimapping("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/100_long_single_reads_cov.sam", "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/bwa-mem2/100_long_sing_end_reads_bwa-mem.sam")

print(f"Alignment accuracy: {accuracy * 100}%")
print(f"Percentage of multi-mapped reads: {multimapping_percentage * 100}%")


# Paired-end Reads Comparison

In [ ]:
reads_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/100_single_end/100_long_single_reads_cov.fq" 

reference_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna"

#output_file = "/home/lajoyce/Documents/sequence_alignment/complete_genome/sam_files/100_single_ends_strobe.sam"


# Define the StrobeAlign command 
# If you want to run strobealign without the predefined index, don't add the "--use-index" parameter
#strobealign_command = ["/home/lajoyce/Documents/sequence_alignment/strobealign/build/strobealign", "--use-index", reference_file, reads_file]

# Command to use if you want to keep the output of strobealign
#If you want to save the output, add the directory and name of the output file
bwamem2_command = ' '.join(["/home/lajoyce/Documents/sequence_alignment/bwa/bwa",
                            "mem",
                            "GCF_009914755.1_T2T-CHM13v2.0_genomic.fna",
                            "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/paired_end_reads/100_paired_end_reads1.fq",
                            "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/paired_end_reads/100_paired_end_reads2.fq"
                             "> /home/lajoyce/Documents/sequence_alignment/complete_genome/trial/bwa-mem2/paired_end_reads.sam"])
                            
# Start the timer

start_time = time.time()

#try:
#Run StrobeAlign and do not discard the output
subprocess.run(bwamem2_command, check=True, shell=True)

#except subprocess.CalledProcessError as e:
#    print("Error:")
 #   print(e.stderr)

#Run StrobeAlign and discard the output since we may only care about about the runtime

#subprocess.run(strobealign_command, stdout=subprocess.DEVNULL, check = True)

# Stop the timer
end_time = time.time()

#Delete the temporary FASTQ file
#os.remove(temp_file_name)

# Calculate and return the runtime 
runtime = end_time - start_time

print(f"Runtime: {runtime} seconds")

In [ ]:
def calculate_accuracy_and_multimapping(ground_truth_sam, test_sam):
    # Initialize variables
    total_reads = 0
    correct_alignments = 0
    multimapped_reads = 0

    with pysam.AlignmentFile(ground_truth_sam, "r") as gt_sam, pysam.AlignmentFile(test_sam, "r") as t_sam:
        for truth, test in zip(gt_sam, t_sam):
            # Increase total reads
            total_reads += 1
            
            # Check if alignments are the same
            if truth.reference_start == test.reference_start:
                correct_alignments += 1

            # Check if read is multi-mapped
            if test.has_tag("XA"):
                multimapped_reads += 1
                
    # If no reads found, print a message and return
    if total_reads == 0:
        print("No reads found in the SAM files.")
        return 0, 0
    
    # Calculate the accuracy
    accuracy = correct_alignments / total_reads
    multimapping_percentage = multimapped_reads / total_reads
    print(f"The number of correct alignments: {correct_alignments}")

    return accuracy, multimapping_percentage

accuracy, multimapping_percentage = calculate_accuracy_and_multimapping("/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/reads_simulated/paired_end_reads/100_paired_end_reads.sam", "/home/lajoyce/Documents/sequence_alignment/complete_genome/trial/bwa-mem2/paired_end_reads.sam")

print(f"Alignment accuracy: {accuracy * 100}%")
print(f"Percentage of multi-mapped reads: {multimapping_percentage * 100}%")